In [1]:
from langchain_core.prompts.chat import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.runnables import RunnableConfig
import query_tool
from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain.memory import ConversationBufferMemory


In [21]:
tools = [
   query_tool.reference_code_search
  ]

message_history = RedisChatMessageHistory(
    url="redis://localhost:6379/0", ttl=600, session_id="my-session"
)

memory = ConversationBufferMemory(
    memory_key="chat_history", chat_memory=message_history, return_messages=True, human_prefix="human", ai_prefix="ai", input_key="input", output_key="output"
)

prompt = ChatPromptTemplate.from_messages([
  ("system", "You are an AWS solutions architect. Your job is to help customers build and deploy secure applications on AWS. Follow architecture best practices and find out approriate requirements before suggesting a solution. Only call tools once necessary for generating a final terraform file."),
  ("human", "My current terraform file:\n{tf_file}"),
  MessagesPlaceholder("chat_history", optional=True),
  ("human", "{input}"),
  "{agent_scratchpad}"
])


llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=.5)
# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(llm, tools, prompt) # type: ignore # wrong type for query_tool import

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory, max_iterations=2, early_stopping_method="generate", return_intermediate_steps=True) # type: ignore # wrong type for query_tool import

In [ ]:
# agent_executor.run({
#   "input": "Hi",
#     "tf_file": "```terraform resource \"aws_instance\" \"example\" { ami = \"${data.aws_ami.ubuntu.id}\" instance_type = \"t2.micro\" }```",
# })

In [22]:
agent_executor.invoke({
  "input": "Which instance type is my provided tf file currently using? dont use any tools.",
  "tf_file": "```terraform resource \"aws_instance\" \"example\" { ami = \"${data.aws_ami.ubuntu.id}\" instance_type = \"t2.micro\" }```",
})




> Entering new AgentExecutor chain...
The instance type used in the provided Terraform file is "t2.micro".

> Finished chain.


{'input': 'Which instance type is my provided tf file currently using? dont use any tools.',
 'tf_file': '```terraform resource "aws_instance" "example" { ami = "${data.aws_ami.ubuntu.id}" instance_type = "t2.micro" }```',
 'chat_history': [],
 'output': 'The instance type used in the provided Terraform file is "t2.micro".',
 'intermediate_steps': []}